In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read files
health_camp_details = pd.read_csv("./Train_2/Train/Health_Camp_Detail.csv")
patient_profile = pd.read_csv("./Train_2/Train/Patient_Profile.csv")
first_health_camp = pd.read_csv("./Train_2/Train/First_Health_Camp_Attended.csv")
second_health_camp = pd.read_csv("./Train_2/Train/Second_Health_Camp_Attended.csv")
third_health_camp = pd.read_csv("./Train_2/Train/Third_Health_Camp_Attended.csv")

train = pd.read_csv("./Train_2/Train/Train.csv")

In [3]:
test = pd.read_csv("./test_l0Auv8Q.csv")

# EDA

In [4]:
train.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [5]:
test.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,505701,6548,21-May-06,1,0,0,0,2
1,500633,6584,02-Jun-06,0,0,0,0,0
2,506945,6582,10-Aug-06,0,0,0,0,0
3,497447,6551,27-Aug-06,0,0,0,0,0
4,496446,6533,19-Sep-06,0,0,0,0,0


In [6]:
health_camp_details.head()

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2
2,6544,03-Nov-03,15-Nov-03,First,F,1
3,6585,22-Nov-03,05-Dec-03,First,E,2
4,6561,30-Nov-03,18-Dec-03,First,E,1


In [7]:
patient_profile.head()

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry
2,508307,0,0,0,0,3,87,46,02-Nov-02,D,BFSI
3,512612,0,0,0,0,1,75,47,02-Nov-02,D,Education
4,521075,0,0,0,0,3,None,80,24-Nov-02,H,Others


In [8]:
print(patient_profile["Patient_ID"].nunique())
print(train["Patient_ID"].nunique())
print(test["Patient_ID"].nunique())

37633
29828
15324


In [9]:
first_health_camp.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN
2,518680,6560,10,0.048780,NaN
3,509916,6560,30,0.634146,NaN
4,488006,6560,20,0.024390,NaN


In [10]:
second_health_camp.head()

,Patient_ID,Health_Camp_ID,Health Score
0,526631,6536,0.875136
1,509122,6536,0.755700
2,498864,6536,0.673181
3,515398,6536,0.722041
4,504624,6536,0.464712


In [11]:
third_health_camp.head()

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1
2,504692,6527,3,1
3,493167,6527,4,4
4,510954,6528,2,2


In [44]:
third_health_camp[third_health_camp["Number_of_stall_visited"]==0]

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
364,486530,6527,0,0
439,492766,6527,0,0
464,510061,6527,0,0
755,514624,6527,0,0
777,506873,6527,0,0
922,489505,6527,0,0
997,503497,6527,0,0
1373,514108,6527,0,0
1381,515663,6527,0,0
1452,516582,6527,0,0


In [75]:
# create a new dataframe to play around with the data without affecting source
working_data = train.copy()

In [76]:
def merge(working_data):
    """To merge health camp details and profile info"""
    a=pd.merge(working_data,health_camp_details,on="Health_Camp_ID",how="left") #merge health camp details
    b=pd.merge(a, patient_profile,on="Patient_ID",how="left") #merge profile info of patient
    return b

In [77]:
def merge_healthscore(cols):
    """Puts 1 if health score exist 0 if not"""
    ID = cols[0]
    health_ID = cols[1]
    c_type = cols[2]
    score = 0
    
    if c_type =='First':
        if ID in first_health_camp["Patient_ID"].values:
            if health_ID in first_health_camp[first_health_camp["Patient_ID"]==ID]["Health_Camp_ID"].values:
                    score=1
        
    elif c_type =='Second':
        if ID in second_health_camp["Patient_ID"].values:
            if health_ID in second_health_camp[second_health_camp["Patient_ID"]==ID]["Health_Camp_ID"].values:
                    score=1
                    
    elif c_type =='Third':
        score="NA"
    
    return score

In [78]:
def merge_stall(cols):
    """Return 1 if stall visited 0 if not"""
    ID = cols[0]
    health_ID = cols[1]
    c_type = cols[2]
    score = 0
    if c_type == "Third":
        if ID in third_health_camp["Patient_ID"].values:
            if health_ID in third_health_camp[third_health_camp["Patient_ID"]==ID]["Health_Camp_ID"].values:
                a=third_health_camp[third_health_camp["Patient_ID"]==ID]
                b=a[a["Health_Camp_ID"]==health_ID]
                if b["Number_of_stall_visited"].values[0]!=0:
                    score=1
    else:
        score="NA"
    return score

In [79]:
working_data=merge(working_data)

In [80]:
working_data["Health_Score_exists"]=working_data[["Patient_ID","Health_Camp_ID","Category1"]].apply(merge_healthscore,axis=1)

In [81]:
working_data["Stall_visited"]=working_data[["Patient_ID","Health_Camp_ID","Category1"]].apply(merge_stall,axis=1)

In [82]:
# create a column favourable 1 if yes 0 if no
def create_favourable(cols):
    score_exists=cols[0]
    stall_visited=cols[1]
    
    ret=0
    if score_exists !="NA":
        if score_exists==1:
            ret=1
        elif score_exists==0:
            ret=0
    elif score_exists=="NA":
        if stall_visited==1:
            ret=1
        elif stall_visited ==0:
            ret = 0
    return ret

In [83]:
working_data["Favourable"]=working_data[["Health_Score_exists","Stall_visited"]].apply(create_favourable,axis=1)

In [84]:
working_data.drop(["Health_Score_exists","Stall_visited"],axis=1,inplace=True)

In [85]:
working_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75278 entries, 0 to 75277
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Patient_ID         75278 non-null  int64 
 1   Health_Camp_ID     75278 non-null  int64 
 2   Registration_Date  74944 non-null  object
 3   Var1               75278 non-null  int64 
 4   Var2               75278 non-null  int64 
 5   Var3               75278 non-null  int64 
 6   Var4               75278 non-null  int64 
 7   Var5               75278 non-null  int64 
 8   Camp_Start_Date    75278 non-null  object
 9   Camp_End_Date      75278 non-null  object
 10  Category1          75278 non-null  object
 11  Category2          75278 non-null  object
 12  Category3          75278 non-null  int64 
 13  Online_Follower    75278 non-null  int64 
 14  LinkedIn_Shared    75278 non-null  int64 
 15  Twitter_Shared     75278 non-null  int64 
 16  Facebook_Shared    75278 non-null  int64

In [87]:
def time_in_days(cols):
    if cols[0] is not np.NaN:
        start_date=str(cols[0]).split('-')
    else:
        start_date=str(cols[1]).split('-')
        
    end_date = str(cols[1]).split('-')
    
    months={"Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,"Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12}
    def dur(date):
        return int(date[2])*365+months[date[1]]*30+int(date[0])
    return dur(end_date)-dur(start_date)

In [88]:
working_data["time_to_start"]=working_data[["Registration_Date","Camp_Start_Date"]].apply(time_in_days,axis=1)
working_data["duration"]=working_data[["Camp_Start_Date","Camp_End_Date"]].apply(time_in_days,axis=1)
working_data["time_after_first"]=working_data[["Registration_Date","First_Interaction"]].apply(time_in_days,axis=1)

In [89]:
working_data.drop(["Registration_Date","Camp_Start_Date","Camp_End_Date"],axis=1,inplace=True)

In [59]:
working_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75278 entries, 0 to 75277
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Patient_ID         75278 non-null  int64 
 1   Health_Camp_ID     75278 non-null  int64 
 2   Var1               75278 non-null  int64 
 3   Var2               75278 non-null  int64 
 4   Var3               75278 non-null  int64 
 5   Var4               75278 non-null  int64 
 6   Var5               75278 non-null  int64 
 7   Category1          75278 non-null  object
 8   Category2          75278 non-null  object
 9   Category3          75278 non-null  int64 
 10  Online_Follower    75278 non-null  int64 
 11  LinkedIn_Shared    75278 non-null  int64 
 12  Twitter_Shared     75278 non-null  int64 
 13  Facebook_Shared    75278 non-null  int64 
 14  Income             75278 non-null  object
 15  Education_Score    75278 non-null  object
 16  Age                75278 non-null  objec

In [90]:
working_data["City_Type"]=working_data["City_Type"].fillna('B')

In [65]:
working_data["Employer_Category"].unique()

array([nan, 'Others', 'Technology', 'Consulting', 'Software Industry',
       'Manufacturing', 'Education', 'Health', 'Retail', 'BFSI',
       'Transport', 'Broadcasting', 'Telecom', 'Real Estate', 'Food'],
      dtype=object)

In [66]:
working_data.groupby(working_data["Employer_Category"])["Patient_ID"].count()

Employer_Category
BFSI                  845
Broadcasting          185
Consulting           2128
Education            1125
Food                  302
Health                446
Manufacturing         510
Others               2239
Real Estate           361
Retail                535
Software Industry    2358
Technology           3533
Telecom               272
Transport             364
Name: Patient_ID, dtype: int64

In [91]:
working_data["Employer_Category"]=working_data["Employer_Category"].fillna("Technology")

In [92]:
cat1=pd.get_dummies(working_data["Category1"],drop_first=True)
cat2=pd.get_dummies(working_data["Category2"],drop_first=True)

working_data= pd.concat([working_data.drop(["Category1","Category2"],axis=1),cat1,cat2],axis=1)

In [93]:
working_data.rename(columns={'B':'B1'},inplace=True)

In [84]:
working_data["Income"].value_counts()

None    53546
0        8393
1        5346
2        4108
3        2155
4        1034
5         440
6         256
Name: Income, dtype: int64

In [94]:
#fill income none values with mean income
working_data["Income"]=working_data["Income"].apply(lambda x: 1.284 if x=="None" else x)

In [102]:
s=0
c=0
for a in working_data["Education_Score"].values:
    if a!="None":
        s+=float(a)
        c+=1
print("avg education score:{}".format(s/c))

avg education score:77.96219856369159


In [95]:
working_data["Education_Score"]=working_data["Education_Score"].apply(lambda x: 77.96 if x=="None" else x)

In [107]:
s=0
c=0
for a in working_data["Age"].values:
    if a!="None":
        s+=int(a)
        c+=1
print("avg age:{}".format(s/c))

avg age:47.90606777655709


In [96]:
working_data["Age"]=working_data["Age"].apply(lambda x: 48 if x=="None" else x)

In [97]:
working_data.drop("First_Interaction",axis=1,inplace=True)

In [98]:
city_typ = pd.get_dummies(working_data["City_Type"],drop_first=True)
emp_category= pd.get_dummies(working_data["Employer_Category"],drop_first=True)

working_data=pd.concat([working_data.drop(["City_Type","Employer_Category"],axis=1),city_typ,emp_category],axis=1)

In [99]:
working_data["Favourable"].value_counts()

0    54744
1    20534
Name: Favourable, dtype: int64

In [25]:
# preparing test data

In [100]:
test_data=merge(test)

In [101]:
test_data["time_to_start"]=test_data[["Registration_Date","Camp_Start_Date"]].apply(time_in_days,axis=1)
test_data["duration"]=test_data[["Camp_Start_Date","Camp_End_Date"]].apply(time_in_days,axis=1)
test_data["time_after_first"]=test_data[["Registration_Date","First_Interaction"]].apply(time_in_days,axis=1)

In [102]:
test_data.drop(["Registration_Date","Camp_Start_Date","Camp_End_Date"],axis=1,inplace=True)

In [103]:
test_data["City_Type"]=test_data["City_Type"].fillna('B')

In [104]:
test_data["Employer_Category"]=test_data["Employer_Category"].fillna("Technology")

In [105]:
cat1=pd.get_dummies(test_data["Category1"],drop_first=True)
cat2=pd.get_dummies(test_data["Category2"],drop_first=True)

test_data= pd.concat([test_data.drop(["Category1","Category2"],axis=1),cat1,cat2],axis=1)

In [106]:
#fill income none values with mean income
test_data["Income"]=test_data["Income"].apply(lambda x: 1.284 if x=="None" else x)

In [107]:
test_data["Education_Score"]=test_data["Education_Score"].apply(lambda x: 77.96 if x=="None" else x)

In [108]:
test_data["Age"]=test_data["Age"].apply(lambda x: 48 if x=="None" else x)

In [109]:
test_data.drop("First_Interaction",axis=1,inplace=True)

In [110]:
city_typ = pd.get_dummies(test_data["City_Type"],drop_first=True)
emp_category= pd.get_dummies(test_data["Employer_Category"],drop_first=True)

test_data=pd.concat([test_data.drop(["City_Type","Employer_Category"],axis=1),city_typ,emp_category],axis=1)

In [113]:
test_data.insert(20,column="B1",value=np.zeros(35249))

# model building

In [116]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [117]:
x=working_data.drop(["Favourable","Patient_ID","Health_Camp_ID"],axis=1).values
y=working_data["Favourable"].values.reshape([75278,])


In [118]:
from sklearn.preprocessing import StandardScaler

In [119]:
scaler=StandardScaler()

In [120]:
scaler.fit(x)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [121]:
x=scaler.transform(x)

In [122]:
x_test = scaler.transform(test_data.drop(["Patient_ID","Health_Camp_ID"],axis=1))

### Neural Net

In [188]:
model=Sequential()

In [189]:
model.add(Dense(50,activation="relu",kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(50,activation="relu",kernel_initializer='he_normal'))
model.add(Dense(1,activation="sigmoid",kernel_initializer="he_normal"))

In [190]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [191]:
hist=model.fit(x,y,batch_size=32,epochs=20)

Epoch 1/20
75278/75278 [==============================] - 8s 109us/step - loss: 0.4770 - acc: 0.7761
Epoch 2/20
75278/75278 [==============================] - 7s 94us/step - loss: 0.4530 - acc: 0.7924
Epoch 3/20
75278/75278 [==============================] - 6s 83us/step - loss: 0.4469 - acc: 0.7969
Epoch 4/20
75278/75278 [==============================] - 7s 90us/step - loss: 0.4428 - acc: 0.7995
Epoch 5/20
75278/75278 [==============================] - 7s 91us/step - loss: 0.4409 - acc: 0.8010
Epoch 6/20
75278/75278 [==============================] - 7s 93us/step - loss: 0.4378 - acc: 0.8025
Epoch 7/20
75278/75278 [==============================] - 7s 94us/step - loss: 0.4356 - acc: 0.8046
Epoch 8/20
75278/75278 [==============================] - 7s 94us/step - loss: 0.4343 - acc: 0.8054
Epoch 9/20
75278/75278 [==============================] - 7s 89us/step - loss: 0.4314 - acc: 0.8063
Epoch 10/20
75278/75278 [==============================] - 7s 90us/step - loss: 0.4307 - acc: 0.807

In [192]:
pred=model.predict(x_test)

In [193]:
df=pd.DataFrame({"Patient_ID":test["Patient_ID"].values,"Health_Camp_ID":test["Health_Camp_ID"].values,"Outcome":pred.reshape([-1])})

In [194]:
df.to_csv("result.csv",index=False)

In [177]:
from sklearn.metrics import roc_auc_score

In [178]:
roc_auc_score(y,model.predict(x))

0.8609002214844366

### Random Forest

In [132]:
from sklearn.ensemble import RandomForestClassifier

In [133]:
model2 = RandomForestClassifier()

In [134]:
from sklearn.model_selection import GridSearchCV

In [135]:
parameters={'n_estimators':[200,300,400,500],'criterion':['gini','entropy'],'min_samples_leaf':[5,10],'min_samples_split':[1,5,10]}

In [136]:
grid_search = GridSearchCV(estimator=model2,
                          param_grid=parameters,
                          n_jobs=-1,
                          scoring='roc_auc')

In [ ]:
grid_search.fit(x,y)

In [ ]:
best_accuracy=grid_search.best_score_
best_params=grid_search.best_params_
print(best_accuracy)
print(best_params)

In [215]:
model2 = RandomForestClassifier(criterion= 'entropy', 
                                min_samples_leaf=5,
                                min_samples_split=5,
                                n_estimators= 500)

In [216]:
model2.fit(x,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features=40,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [217]:
predict=model2.predict_proba(x_test)[:,1]

In [218]:
df=pd.DataFrame({"Patient_ID":test["Patient_ID"].values,"Health_Camp_ID":test["Health_Camp_ID"].values,"Outcome":predict.reshape([-1])})

In [219]:
df.to_csv("random_forest.csv",index=False)